In [1]:
import pandas as pd
import numpy as np
import time
import pyarrow.parquet as pq
import scipy
#import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
import pycaret.classification as pc

In [2]:
#чтение, вывод инфы
data = pq.read_table('data/competition_data_final_pqt')
#display(pd.DataFrame([(z.name, z.type) for z in data.schema], columns = [['field', 'type']]))
print(len(data))
targets = pq.read_table('data/public_train.pqt')
display(pd.DataFrame([(z.name, z.type) for z in targets.schema], columns = [['field', 'type']]))
print(len(targets))

,field,type
0,region_name,string
1,city_name,string
2,cpe_manufacturer_name,string
3,cpe_model_name,string
4,url_host,string
5,cpe_type_cd,string
6,cpe_model_os_type,string
7,price,double
8,date,date32[day]
9,part_of_day,string


322899435


,field,type
0,age,double
1,is_male,string
2,user_id,int64
3,__index_level_0__,int64


270000


In [3]:
data.select(['cpe_type_cd']).to_pandas()['cpe_type_cd'].value_counts()

smartphone    322781599
tablet            53768
plain             36116
phablet           27952
Name: cpe_type_cd, dtype: int64

In [4]:
%%time
data_agg = data.select(['user_id', 'url_host', 'request_cnt']).group_by(['user_id', 'url_host']).aggregate([('request_cnt', "sum")])

CPU times: total: 15.5 s
Wall time: 35.2 s


In [6]:
data_agg.to_pandas()

,request_cnt_sum,user_id,url_host
0,12,45098,ad.adriver.ru
1,491,45098,apple.com
2,174,45098,avatars.mds.yandex.net
3,305,45098,googleads.g.doubleclick.net
4,446,45098,i.ytimg.com
...,...,...,...
32277664,1,300964,youtube.com
32277665,1,300964,biosfera.kz
32277666,1,300964,chihuahuadog-ru.turbopages.org
32277667,1,300964,sun9-88.userapi.com


In [ ]:
url_set = set(data_agg.select(['url_host']).to_pandas()['url_host'])
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
usr_set = set(data_agg.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

Делаем странные вещи

In [ ]:
%%time
values = np.array(data_agg.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])
rows = np.array(data_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(data_agg.select(['url_host']).to_pandas()['url_host'].map(url_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, calculate_training_loss = False, regularization = 0.1)

In [ ]:
%%time
als.fit(mat)
u_factors = als.model.user_factors
d_factors = als.model.item_factors

типа подготовка данных

In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
usr_emb = pd.DataFrame(u_factors)
usr_emb['user_id'] = usr_emb.index.map(inv_usr_map)
usr_targets = targets.to_pandas()
df = usr_targets.merge(usr_emb, how = 'inner', on = ['user_id'])
df = df[df['is_male'] != 'NA']
df = df.dropna()
df['is_male'] = df['is_male'].map(int)
df['is_male'].value_counts()

каким-то боком еще и кетбуст

In [ ]:
%%time
x_train, x_test, y_train, y_test = train_test_split( \
    df.drop(['user_id', 'age', 'is_male'], axis = 1), df['is_male'], test_size = 0.33, random_state = SPLIT_SEED)
clf = CatBoostClassifier()
clf.fit(x_train, y_train, verbose = False)
print(f'GINI по полу {2 * m.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]) - 1:2.3f}')

С возрастом у них более менее адекватно, есть крутая штука classification_report